<a href="https://colab.research.google.com/github/FrankT990/Sent_Analyzer/blob/main/Sentiment_Analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests
!pip install newsapi-python
!pip install finnhub-python
!pip install scikit-learn
import finnhub
import time
import yfinance as yf
import requests
from datetime import datetime, timedelta
from newsapi import NewsApiClient
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [2]:
# This code block is for functions that help with testing / printing outputs

# uses finnhub api
# Example of req:
# finnhub_client.company_news('SPY', _from="2023-09-04", to="2024-09-04")
def print_headlines_datetimes(req):
  for article in req:
    print(datetime.utcfromtimestamp(article['datetime']).strftime('%Y-%m-%d'))

# How to make df:
# stock = yf.Ticker("SPY")
# hist = stock.history(period="1y")
# df = pd.DataFrame(hist)
def print_SPY_performance_datetimes(df):
  for ts, row in df.iterrows():
    print(ts.strftime('%Y-%m-%d'))

def one_year_ago():
  return (datetime.now() - timedelta(days=365)).strftime('%Y-%m-%d')

def date_now():
  return datetime.now().strftime('%Y-%m-%d')

# unix_time_stamp -> YYYY-MM-DD
def unix_timestamp_to_date(unix_timestamp):
  return datetime.utcfromtimestamp(unix_timestamp).strftime('%Y-%m-%d ')



In [7]:
# from_date and to_date in format of YYYY-MM-DD
# ex: 2024-09-04
def write_headlines(tkr, from_date, to_date, finnhub_api_key):
    try:
      finnhub_client = finnhub.Client(api_key=finnhub_api_key)
      df = pd.DataFrame()
      news = finnhub_client.company_news(tkr, _from=from_date, to=to_date)
      for article in news:
        date = unix_timestamp_to_date(article['datetime'])
        new_row = pd.DataFrame({
        'date': [date],
        'headline': [article['headline']]
        })
        df = pd.concat([df, new_row], ignore_index=True)
      return df
    except:
      return 'err'

def write_performance_dict(tkr):
  try:
    perf_dict = {}
    stock = yf.Ticker(tkr)
    hist = stock.history(period="1y")
    yf_df = pd.DataFrame(hist)
    for ts, row in yf_df.iterrows():
      info = row.to_dict()
      perc_change = (info['Close'] - info['Open']) / info['Open']
      perf_dict[ts.strftime('%Y-%m-%d')] = perc_change
    return perf_dict
  except:
    return "err"

def add_data(perf_dict, headlines_df):
  try:
    df = pd.DataFrame()
    # main_dict = {}
    for i, row in headlines_df.iterrows():
      date = row['date'].strip()
      headline = row['headline']
      pc = perf_dict.get(date, 'Undefined key')
      if(type(pc) == float):
        new_row = pd.DataFrame({
        'headline': [headline],
        'percent_change': [pc]
        })
        # main_dict[headline] = pc
        df = pd.concat([df, new_row], ignore_index=True)
    return df
  except:
    return pd.DataFrame()

def make_model(df):
  # FROM DOCS
  vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
  X = vectorizer.fit_transform(df['headline'])

  y = df['percent_change'].values

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  model = RandomForestRegressor(n_estimators=100, random_state=42)
  model.fit(X_train, y_train)

  y_pred = model.predict(X_test)
  mse = mean_squared_error(y_test, y_pred)

  return [model, vectorizer, mse]


In [23]:

def prepare_df(stocks, finnhub_key):
  df = pd.DataFrame()
  for stock in stocks:
    perf_dict = write_performance_dict(tkr=stock)
    headlines_df = write_headlines(tkr=stock, from_date=one_year_ago(), to_date=date_now(), finnhub_api_key=finnhub_key)
    if (type(perf_dict) != str and type(headlines_df) != str):
      df = pd.concat([df, add_data(perf_dict, headlines_df)], ignore_index=True)
  return df

def get_stock_input():
  print(chr(27) + "[2J")
  default_stocks_list=['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'TSLA', 'NVDA']
  print('Would you like to train the model using your own stock list? (y/n) '+'\n'+'(Note: Acceptable size of stock list is dependent on API key)')
  inp = input()
  match inp:
    case 'y':
      print('Please input stocks in Python list format\nex: [\'AAPL\', \'MSFT\', \'GOOGL\', \'AMZN\', \'META\', \'TSLA\', \'NVDA\']')
      stocks = list(input())
      return stocks
      print('Creating model using custom stock list...')
    case 'n':
      print('Creating model using default stock list...')
      return default_stocks_list
    case default:
      print('Invalid input')
      get_stock_input()

def predict_headline(model, vectorizer):
  print('Input headline to test\n')
  headline = input()
  headline_vector = vectorizer.transform([headline])
  pc = model.predict(headline_vector)
  if pc[0] > 0:
    print('Market Sentiment: Bullish (Estimated percent change: + %' + str(pc[0]*100)+')\n')
  else:
    print('Market Sentiment: Bearish (Estimated percent change: - %' + str(pc[0]*100)+')\n')

def control_flow():
  print('1. Test Another Headline\n2. Re-train model on new stock list\n')
  inp = input()
  match inp:
    case '1':
      predict_headline(data[0], data[1])
    case '2':
      stocks = get_stock_input()
      df = prepare_df(stocks, finnhub_key)
      data = make_model(df)
      print('Model created!\nMSE: '+str(data[2]))
    case default:
      print('Invalid input')
      control_flow()
  control_flow()


In [ ]:
# FRONT END
# Please make sure to run preceeding code blocks this code block
# ""

print('Please input a finnhub API key: \n(You can get one for free at https://finnhub.io/)')
finnhub_key = input()
stocks = get_stock_input()
df = prepare_df(stocks, finnhub_key)
data = make_model(df)
print('Model created!\nMSE: '+str(data[2]))
predict_headline(data[0], data[1])
control_flow()